In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from math import sin, cos, radians

# 1 Analyze Uncertainty of Vectors Projected by the Test System

In [2]:
# Test system parameters
# W = 110.69009321481776 # active panel width, mm
# H = 62.263177433334995 # active panel height, mm

Res_W = 1920 # width resolution, pixels
Res_H = 1080 # height resolution, pixels 
f_c = 350 # focal length of the collimating lens, mm

pixel_size = 0.05765109021605092 # mm


u0 = (Res_W/2) * pixel_size
v0 = (Res_H/2) * pixel_size

In [3]:
rng = np.random.default_rng()

In [4]:
center_array = np.zeros((Res_H, Res_W,3))
for row in range(Res_H):
    for col in range(Res_W):
        u = (col+0.5)*pixel_size
        v = (row+0.5)*pixel_size
        center_array[row,col,0] = u0-u
        center_array[row,col,1] = v0-v
        center_array[row,col,2] = f_c

center_array_norm = center_array/np.linalg.norm(center_array,axis=2, keepdims=True) 

In [5]:
sample_num = 250
angle_list = []
for i in trange(sample_num):

    offset = rng.uniform(-0.5,0.5,(Res_H,Res_W,2))*pixel_size

    offset_array = center_array.copy() #!!!! should add offset to unnormalized vectors !!!!#
    offset_array[:,:,:2] += offset 
    offset_array /= np.linalg.norm(offset_array,axis=2,keepdims=True)

    dot_product = np.sum(offset_array*center_array_norm,axis=2)

    dot_product = np.clip(dot_product,-1,1)
    
    angle = np.rad2deg(np.arccos(dot_product))

    angle_list.extend(list(angle.flatten()))




  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [01:42<00:00,  2.43it/s]


: 

In [ ]:
angle_array = np.array(angle_list)
angle_mean = np.mean(angle_array)
angle_sigma = np.std(angle_array)

In [6]:
print(f'mean={angle_mean*3600}arcsec, sigma={angle_sigma*3600}arcsec')


KeyboardInterrupt: 

# 2 Angular Distance Error Between a Pair of Stars

In [ ]:
def euler_axis_angle(e, theta):
    ex = np.array(  [0, -e[2], e[1]], 
                    [e[2], 0, -e[0]],
                    [-e[1], e[0],0])
    A = np.eye(3) - sin(radians(theta))*ex + (1-cos(radians(theta)))*ex@ex
    return A
